In [1]:
import numpy as np
import re
import nltk

data = np.load('data/corpus_CleanNewsFilter.pkl', allow_pickle=True)

In [2]:
stopwords = nltk.corpus.stopwords.words('english')
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [3]:
article = data[1]
print(article)

Arthur J. Gallagher & Co., together with its subsidiaries, provides insurance brokerage, consulting, and third party claims settlement and administration services to entities in the United States and internationally. Its Brokerage segment consists of retail and wholesale insurance brokerage operations. It offers brokerage and consulting services to businesses and organizations, including commercial, not-for-profit, and public entities, as well as individuals in the areas of insurance placement, risk of loss management, and management of employer sponsored benefit programs. This segment also assist retail brokers and other non-affiliated brokers in the placement of specialized and hard-to-place insurance; acts as a brokerage wholesaler, and managing general agent or managing general underwriter distributing specialized insurance coverage's for underwriting enterprises; and performs activities, including marketing, underwriting, issuing policies, collecting premiums, appointing and super

In [39]:
stopwords = nltk.corpus.stopwords.words('english')

def get_sentences(article):
    '''
        pipline
    '''
    # TODO Firstly, convert Chinese punctuations into English ones

    # remove "." in abbreviation, "U.K" -> "UK"
    text = re.sub(r'(?<!\w)([A-Z])\.', r'\1', article)

    # remove extra space
    text = re.sub(" +", " ", text)

    # remove "," in numbers
    p = re.compile("\d+,\d\d\d")
    for com in p.finditer(text):
        mm = com.group()
        text = text.replace(mm, mm.replace(",", ""))

    # nltk cut sentences
    text = [s for s in nltk.tokenize.sent_tokenize(text)]

    # Replace periods and commas with spaces
    blocklist = ["'s", "’s", "news record", "\n", "\xa0"]
    res = []
    for s in text:
        s = s.lower()
        for i in blocklist:
            s = re.sub(f'{i}', ' ', s)
        s = re.sub(r'[\W]',' ',s)
        s = re.sub(" +", " ", s).strip()  # extra space
        if len(s)>20:  # save sentences which have more than 20 words
            s = s.split(' ')
            s_new = []
            for w in s:
                if w not in stopwords:  # stop words
                    s_new.append(w)
            res.append(s_new)

    return res

In [40]:
' '.join(get_sentences(data[1])[4])

'company risk management segment provides contract claim settlement administration services enterprises public entities claims management loss control consulting insurance property appraisal services'

In [1]:
# check the word2vec input data

import numpy as np

sentences = np.load('data/words_CleanNewsFilter.pkl', allow_pickle=True)

In [5]:
' '.join(sentences[9])

'accomplishments include winning significant federal case age 26 changing course trade secret law intersection antitrust intellectual property law leading largest us healthcare ipo took avantor initial public offering ipo new york stock exchange may 2019'

In [2]:
# multi-word expressions

from tqdm import tqdm

co_dict = {}

for sentence in tqdm(sentences):
    for i in range(len(sentence)-1):
        word = sentence[i] + '_' + sentence[i+1]
        co_dict[word] = co_dict.get(word, 0) + 1


100%|██████████| 9367772/9367772 [02:37<00:00, 59555.51it/s]


In [4]:
co_dict_sorted = sorted(co_dict.items(),key=lambda x:x[1], reverse=True)

In [5]:
co_dict_sorted

[('per_share', 295355),
 ('company_stock', 269355),
 ('fourth_quarter', 189162),
 ('buy_rating', 188663),
 ('covid_19', 175285),
 ('shares_company', 170047),
 ('free_report', 161781),
 ('year_year', 149422),
 ('consensus_estimate', 147748),
 ('earnings_per', 142615),
 ('stock_valued', 141741),
 ('forward_looking', 138233),
 ('price_target', 137788),
 ('last_quarter', 135194),
 ('average_price', 131849),
 ('hold_rating', 124850),
 ('shares_last', 121392),
 ('moving_average', 120368),
 ('looking_statements', 119229),
 ('zacks_rank', 119139),
 ('first_quarter', 117579),
 ('non_gaap', 114074),
 ('target_price', 113257),
 ('last_year', 110859),
 ('third_quarter', 107056),
 ('inc_nasdaq', 105165),
 ('hedge_funds', 104388),
 ('stock_worth', 104263),
 ('research_report', 103926),
 ('research_note', 101835),
 ('united_states', 96261),
 ('institutional_investors', 96096),
 ('long_term', 95289),
 ('4th_quarter', 93340),
 ('zacks_consensus', 91032),
 ('new_york', 88937),
 ('securities_exchange', 8


---

### TODO
1. Not cleaning enough, "3_2", "10_q", "0_04", maybe because of float numbers.
2. Three or four or more words multi-word expression, "COVID_19_pandemic", 
3. The final step in processing the data , to connect multi-word expression by "_",  the strategy for adding multi-word expression ?